In [2]:
import random
import collections
import math
import sys
import numpy as np
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr


##KEY CODE

def importTickerNames():
    #http://www.nasdaqtrader.com/trader.aspx?id=symboldirdefs
    nasdaqData = pd.read_csv('nasdaqlisted.txt', sep = "|")
    nasdaqData = nasdaqData[nasdaqData["ETF"] == "N"]
    nasdaqList  = nasdaqData["Symbol"].tolist()
    nasdaqList = nasdaqList[:len(nasdaqList)-1]
    otherData = pd.read_csv('otherlisted.txt', sep = "|")
    otherData = otherData[otherData["ETF"] == "N"]
    otherList = otherData["ACT Symbol"].tolist() 
    symbolList = nasdaqList + otherList
    symbolList.sort()
    with open('tickerList.txt', 'w') as filehandle:
        filehandle.writelines("%s\n" % symbol for symbol in symbolList)
    return symbolList
   

def getPERMNO():
    data = pd.read_csv('pricesData.csv', sep = ",")
    permnoList = data["PERMNO"].to_list()
    permnoList = list(dict.fromkeys(permnoList))
    with open('permnoList.txt', 'w') as filehandle:
        filehandle.writelines("%s\n" % permno for permno in permnoList)
    return permnoList

def bullMarketData(data):
    #https://www.cnbc.com/2020/03/14/a-look-at-bear-and-bull-markets-through-history.html
    startDates = ["1932/06/01", "1942/04/22", "1949/06/14", "1957/10/22", "1962/06/27", "1966/10/07", "1970/05/26", "1974/10/3", "1982/08/12", "1987/12/04", "1990/10/11", "2002/10/09", "2009/03/09"]
    endDates = ["1937/03/05", "1946/05/28", "1956/08/01", "1961/12/11", "1966/02/08", "1968-11-28", "1973/01/10", "1980/11/27", "1987/08/24", "1990/07/15", "2000/03/23", "2007/10/08", "2020/02/18"]
    data = data[((data["date"] > startDates[5]) & (data["date"] < endDates[5])) | ((data["date"] > startDates[6]) & (data["date"] < endDates[6]))| ((data["date"] > startDates[7]) & (data["date"] < endDates[7]))| ((data["date"] > startDates[8]) & (data["date"] < endDates[8]))| ((data["date"] > startDates[9]) & (data["date"] < endDates[9]))| ((data["date"] > startDates[10]) & (data["date"] < endDates[10]))| ((data["date"] > startDates[11]) & (data["date"] < endDates[11]))| ((data["date"] > startDates[12]) & (data["date"] < endDates[12]))]
    return data

def bearMarketData(data):
    #https://www.cnbc.com/2020/03/14/a-look-at-bear-and-bull-markets-through-history.html
    startDates = ["1937/03/06", "1946/05/29", "1956/08/02", "1961/12/12", "1966/02/09", "1968/11/29", "1973/01/11", "1980/11/28", "1987/08/25", "1990/07/16", "2000/03/24", "2007/10/09", "2020/02/19"]
    endDates = ["1942/04/28","1949/06/13", "1957/10/21", "1962/06/26", "1966/10/06", "1970/05/25", "1974/10/2", "1982/08/11", "1987/12/03", "1990/10/10", "2002/10/08", "2009/03/08", "2020/04/21"]
    data = data[((data["date"] > startDates[5]) & (data["date"] < endDates[5])) | ((data["date"] > startDates[6]) & (data["date"] < endDates[6]))| ((data["date"] > startDates[7]) & (data["date"] < endDates[7]))| ((data["date"] > startDates[8]) & (data["date"] < endDates[8]))| ((data["date"] > startDates[9]) & (data["date"] < endDates[9]))| ((data["date"] > startDates[10]) & (data["date"] < endDates[10]))| ((data["date"] > startDates[11]) & (data["date"] < endDates[11]))| ((data["date"] > startDates[12]) & (data["date"] < endDates[12]))]
    return data 

def getData():
    #prices & volume: https://wrds-web-wharton-upenn-edu.stanford.idm.oclc.org/wrds/ds/wrdsapps/finratiofirm/index.cfm?navId=83
    pricesData = pd.read_csv('pricesData.csv', sep = ",")
    financialRatiosData = pd.read_csv('financialRatiosData.csv', sep = ",")
    return pricesData, financialRatiosData

def addMomentumFactor(pricesData):
    #Add Momentum Factor 
    momentum = list(pricesData["PRC"])
    #print(([0] + momentum)[:10])
    #print(momentum[:10])
    momentum = np.divide(np.subtract(momentum , [0] + momentum[:-1]), [0] + momentum[:-1])
    pricesData["Momentum"] = momentum
    '''
    prevNum = 0
    for i in range(0, len(pricesData["Momentum"])):
        if pricesData["PERMNO"][i] > prevNum:
            prev = pricesData["PERMNO"][i]
            pricesData["Momentum"][i] = float("nan")
    print(pricesData.head())
    '''
    
def mergeData(pricesData, financialRatiosData):
    dates = list(pricesData["date"])
    monthList = []
    for date in dates:
        monthList.append(date[:7])
    pricesData["Month"] = monthList
    
    dates = financialRatiosData["public_date"].to_list()
    monthList = []
    for date in dates:
        monthList.append(date[:7])
    financialRatiosData["Month"] = monthList
    
    totalData = pd.DataFrame(columns = list(pricesData.columns) + list(financialRatiosData.columns)[4:-1])
    nums = np.unique(list(pricesData["PERMNO"]))
    for num in nums:
        numData1 = pricesData[pricesData["PERMNO"] == num]
        numData2 = financialRatiosData[financialRatiosData["permno"] == num]
        numData2 = numData2.drop(['permno', 'adate', 'qdate', 'public_date'], axis = 1)
        result = pd.merge(numData1, numData2, on = "Month")
        totalData = pd.concat([totalData,result])
        print(num)
    return totalData
    
    
#EXTRA CODE
'''
def financialsIntoTiles(data, n):
    financials = data
    for column in list(data.columns):
        financials[column] = financials[column].fillna(0)
        financials[column] = pd.qcut(data[column],n,labels=False, duplicates = 'drop')
    return financials
'''

"\ndef financialsIntoTiles(data, n):\n    financials = data\n    for column in list(data.columns):\n        financials[column] = financials[column].fillna(0)\n        financials[column] = pd.qcut(data[column],n,labels=False, duplicates = 'drop')\n    return financials\n"

In [3]:
#Data Cleaning
data = pd.read_csv('totalData.csv', sep = ",")
removeColumnTotal = ["PEG_trailing", "DIVYIELD","PEG_ltgforward", "dpr", "pe_op_basic", "fcf_ocf"]
data = data.drop(columns= removeColumnTotal)
removeColumnNAs = ["TICKER","invt_act", "CAPEI","pe_inc","bm"]
data = data.dropna(how='any', subset= removeColumnNAs)

In [4]:
#Spilt Data
bearData = bearMarketData(data)
bullData = bullMarketData(data)
trainBullData = bullData[bullData["date"] < "2007/10/08"]
testBullData = bullData[bullData["date"] > "2007/10/08"]


In [5]:
#Save files
trainBullData.to_csv('trainBullData.csv')
testBullData.to_csv('testBullData.csv')
bearData.to_csv('bearData.csv')

In [217]:
trainBullData = pd.read_csv("trainBullData.csv")
trainBullData = trainBullData.sort_values('date')
testBullData = pd.read_csv("testBullData.csv")
testBullData = testBullData.sort_values('date')
#Normalization
colToNorm = ["PRC", 'VOL','Momentum', 'CAPEI', 'bm', 'evm', 'pe_inc', 'ps', 'pcf','ptpm', 'roa', 'roe', 'roce', 'aftret_invcapx', 'aftret_equity','GProf', 'equity_invcap', 'debt_invcap', 'capital_ratio', 'invt_act','rect_act', 'cash_debt', 'dltt_be', 'de_ratio', 'cash_ratio','quick_ratio', 'curr_ratio', 'rd_sale', 'ptb']
for stock in trainBullData["PERMNO"].unique():
    stockData = trainBullData[trainBullData["PERMNO"] == stock]
    stockData[colToNorm] = stockData[colToNorm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    trainBullData[trainBullData["PERMNO"] == stock] = stockData
for stock in testBullData["PERMNO"].unique():
    stockData = testBullData[testBullData["PERMNO"] == stock]
    stockData[colToNorm] = stockData[colToNorm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    testBullData[testBullData["PERMNO"] == stock] = stockData